In [1]:
%pylab inline

import gc
import os
import pickle
import h5py
import json
import random
import time

import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.options.mode.chained_assignment = None

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataset = pd.read_csv('../data/MSK_train.csv', sep = ';')

In [3]:
dataset.columns = ['ID', 'TS',
                  'BID0', 'VBID0', 'BID1', 'VBID1', 'BID2', 'VBID2', 'BID3', 'VBID3', 'BID4', 'VBID4', 
                  'BID5', 'VBID5', 'BID6', 'VBID6', 'BID7', 'VBID7', 'BID8', 'VBID8', 'BID9', 'VBID9',
                  'ASK0', 'VASK0', 'ASK1', 'VASK1', 'ASK2', 'VASK2', 'ASK3', 'VASK3', 'ASK4', 'VASK4',
                  'ASK5', 'VASK5', 'ASK6', 'VASK6', 'ASK7', 'VASK7', 'ASK8', 'VASK8', 'ASK9', 'VASK9', 'Y']

In [4]:

tea = dataset[dataset.ID == 'TEA']
cofe=dataset[dataset.ID=='COFFEE']
yall = np.array(tea.Y)

In [ ]:

v=tea
p=pd.DataFrame()
for i in range(9):
    p['diff'+str(i)]=v['ASK'+str(i)]-v['BID'+str(i)]
#p = pd.concat([p,tea[['ASK0','BID0','ASK1','BID1']]], axis =1)
Xall = p.values
lenall = len(Xall)

In [5]:
c=cofe
mid_price=(c['BID0']+c['ASK0'])*0.5
yc=pd.Series(mid_price).rolling(window=100).std().shift(-99)
yc=yc.dropna(axis=0,how='any')
yc = yc.values


In [ ]:
#last tom
v=tea
p=pd.DataFrame()
for i in range(9):
    p['diff'+str(i)]=v['ASK'+str(i)]-v['BID'+str(i)]
for i in range(9):
    p['var'+str(i)]=abs(v['VASK'+str(i)]-v['VBID'+str(i)])

cp=pd.DataFrame()
for i in range(9):
    cp['diff'+str(i)]=c['ASK'+str(i)]-c['BID'+str(i)]
for i in range(9):
    cp['var'+str(i)]=abs(c['VASK'+str(i)]-c['VBID'+str(i)])
cp=cp.iloc[:len(yc)]
xall=pd.concat([p,cp])
Xall = xall.values
lenall = len(Xall)

In [6]:
#new
v=tea
p=pd.DataFrame()
for i in range(9):
    
    p['diff'+str(i)]=(v['ASK'+str(i)].rolling(window=1000).mean().shift(-999)-
                      v['BID'+str(i)].rolling(window=1000).mean().shift(-999))
for i in range(9):
#    p['var'+str(i)]=abs(v['VASK'+str(i)].rolling(window=1000).mean().shift(-999)-
#                      v['VBID'+str(i)].rolling(window=1000).mean().shift(-999))
    p['var'+str(i)]=abs(v['VASK'+str(i)]-v['VBID'+str(i)])
p = p.dropna(axis=0,how='any')


In [7]:

cp=pd.DataFrame()
for i in range(9):
    cp['diff'+str(i)]=(c['ASK'+str(i)].rolling(window=1000).mean().shift(-999)-
                      c['BID'+str(i)].rolling(window=1000).mean().shift(-999))
for i in range(9):
    cp['var'+str(i)]=abs(c['VASK'+str(i)]-c['VBID'+str(i)])
cp=cp.iloc[:len(yc)]
xall=pd.concat([p,cp])
Xall = xall.values

yall = yall[:len(p.values)]




lenall = len(Xall)

In [8]:
yall = np.append(yall,yc)

In [9]:
Xall[np.isnan(Xall)] = 0.
yall[np.isnan(yall)] = 0.

In [10]:
assert(0 == np.count_nonzero(np.isnan(Xall)))
assert(0 == np.count_nonzero(np.isnan(yall)))

In [11]:
lgbparam                 = {}
lgbparam['metric']       = 'rmse'
lgbparam['application']  = 'regression'
lgbparam['nthread']      = 4
lgbparam['seed']         = 1
lgbparam['max_depth']         = 3
#lgbparam['learning_rate'] = 0.01
#lgbparam['boosting']         = 'dart'
#lgbparam['free_raw_data']         = False
#lgbparam['n_estimators'] = 20
lgbnumround              = 100

In [12]:
sep = int(0.5 * lenall)

In [13]:
print(sep, lenall - sep)

(7792562, 7792562)


In [14]:
lgbfull = lgb.Dataset(Xall[:], label=yall[:])
lgbtrain = lgb.Dataset(Xall[:sep], label=yall[:sep])
lgbtest  = lgb.Dataset(Xall[sep:], label=yall[sep:])

In [15]:
def rmse(ytrue, ypred):
    return np.sqrt(mean_squared_error(ytrue, ypred))

In [ ]:
model = lgb.train(lgbparam, lgbtrain, lgbnumround, valid_sets=[lgbtrain, lgbtest], verbose_eval=10)
ypred = model.predict(Xall)

if sep != 0:
    score1 = rmse(yall[:sep], ypred[:sep])
    score2 = rmse(yall[sep:], ypred[sep:])
    
    print(score1)
    print(score2)

In [16]:
model_filename = 'our_model.txt'

In [ ]:
# Save pre-trained model
model.save_model(model_filename)

In [17]:
#fine train
model = lgb.train(lgbparam, lgbfull, lgbnumround, valid_sets=[lgbtrain, lgbtest], verbose_eval=10)

[10]	valid_0's rmse: 0.195122	valid_1's rmse: 0.208989
[20]	valid_0's rmse: 0.190387	valid_1's rmse: 0.205107
[30]	valid_0's rmse: 0.189292	valid_1's rmse: 0.204191
[40]	valid_0's rmse: 0.188855	valid_1's rmse: 0.203857
[50]	valid_0's rmse: 0.188566	valid_1's rmse: 0.203684
[60]	valid_0's rmse: 0.18843	valid_1's rmse: 0.203505
[70]	valid_0's rmse: 0.188205	valid_1's rmse: 0.203383
[80]	valid_0's rmse: 0.187992	valid_1's rmse: 0.203292
[90]	valid_0's rmse: 0.187736	valid_1's rmse: 0.203219
[100]	valid_0's rmse: 0.187636	valid_1's rmse: 0.203137


In [18]:
model.save_model(model_filename)

In [ ]:
# Load pre-trained model
loaded_model = lgb.Booster(model_file=model_filename)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'max_depth': [3, 5, 7, 10, 15 ,20,30, 50]
}

gbm = GridSearchCV(estimator, param_grid, scoring = 'mean_squared_error')

gbm.fit(Xall, yall)

In [ ]:
gbm.best_params_